### Disclaimer
* Catboost package versiyon ve thread counttan ötürü sonuçlarda çok ufak değişiklikler olabilir. Ne kadar eşitlemeye çalışsam da tam benzer sonucu alamadım.
* Discussionda da belirtildiği üzere bizlerden belirli varsayımların belirlenmesi istendi. Bu çalışmadaki varsayımlar şöyledir:
    * [EPİAŞ Şeffaflık](https://seffaflik.epias.com.tr/transparency/) Platformundan alınan bütün veriler minumum lag(24) yani bir gün gecikmeli olarak kullanıldı. Kod ve skor üzerinden de teyit edilebilir.
    * Hava durumu verisi aynı timestamp ile kullanıldı. Bunun nedeni günlük hava durumu tahmini içeren geriye dönük bir verisetini ücretsiz olarak bulamamış olmam ve aslında short-term (bu çalışmada 1 gün) hava durumu tahminlerinin oldukça başarılı sonuç vermesidir.
    * Herhangi bir şekilde forward leak (-1,-2,-3,...,-n) kullanılmamıştır.
    

Nihai Çözümde Kullanılan Dış Veriler:
* Resmi/Dini/Milli Tatiller
* NASA Hava Durumu (Temperature, Wind, Humidty)
* Meteostat Hava Durumu (NASA ile benzer)
* Güneş Işınları (Solar irradiance)
* EPİAŞ Şeffaflık Real Time Consumption - Minumum 24 lag (Türkiye'nin toplam enerji tüketimi)
* EPİAŞ Şeffaflık Real Time Generation - Minumum 24 lag (Türkiye'nin toplam enerji üretimi)

In [1]:
# pip install pvlib #Solar featurelar için gerekli kütüphane

In [2]:
# pip install meteostat #Hava durumu verisi python API kütüphanesi

In [3]:
import warnings
import requests
import pvlib
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import json
import gc
import datetime
from tqdm.notebook import tqdm
from sklearn.preprocessing import SplineTransformer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error
from pvlib.location import Location
from pvlib import clearsky, solarposition,tracking
from meteostat import Hourly,Point
from datetime import datetime
from catboost import CatBoostRegressor

tqdm.pandas()
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

In [4]:
class CFG:
    """
    Class'ın amacı:
    - Feature engineering aşamasını aç-kapa formatına dönüştürerek bug azaltmak
    - Nihai çözümümde bazı featurelar olmasa dahi deneme yapılan feature kapsamını genişletmek/gösterebilmek.
    """
    
    target = 'Dağıtılan Enerji (MWh)'
    train_path = 'C:\\Users\\Emincan\\Desktop\\Competition-Ensembles\\Gdz\\1st-Solution\\train.csv'
    submission_path = 'C:\\Users\\Emincan\\Desktop\\Competition-Ensembles\\Gdz\\1st-Solution\\sample_submission.csv'
    med_path = 'C:\\Users\\Emincan\\Desktop\\Competition-Ensembles\\Gdz\\1st-Solution\\med.csv'
    holiday_path = 'C:\\Users\\Emincan\\Desktop\\Competition-Ensembles\\Gdz\\1st-Solution\\externals\\Calendar.csv'
    read_from_path = True
    
    #Hangi dataların kullanılacağına karar verme
    production_features = True
    consumption_features = False
    yuk_tahmin_plani_features = False
    kgup_features = False
    nasa_weather_features = True
    meteostat_weather_features = True
    holiday_features = True
    solar_features = True
    seasonality_features = True
    seasonality_spline_features = True
    
    
    #Feature türlerine karar verme
    production_lag_features = True
    consumption_lag_features = False
    diff_pct_features = False
    weather_lag_features = True
    solar_lag_features = True
    rolling_features = False
    rolling_shift_features = False
    
    #Feature parametreleri
    nasa_feature_columns =  ['T2M','T2MDEW','T2MWET','QV2M','RH2M','PRECTOTCORR','PS','WS10M','WD10M','WS50M']
    meteostat_feature_columns = ['dwpt','rhum','prcp','wdir','wspd','pres','coco']
    production_base_columns = ['fueloil','gasOil','blackCoal','lignite','geothermal','naturalGas','river','dammedHydro','lng','biomass','importCoal','asphaltiteCoal','wind','sun','importExport','wasteheat','total']
    consumption_base_columns = ['consumption']
    weather_lag_range = np.arange(0,51,5)
    rolling_range = np.arange(0,48,6)[1:]
    roll_types = ['mean','std','min','max']

### Read Base Data

In [5]:
df = pd.read_csv(CFG.train_path)
ss = pd.read_csv(CFG.submission_path)
med = pd.read_csv(CFG.med_path)
med['kesinti_flag'] = 1
med = med.rename(columns={'Tarih':'Tarih_daily'})
med['Tarih_daily'] = pd.to_datetime(med['Tarih_daily'])
df['data'] = 'train'
ss['data'] = 'sub'

df = pd.concat([df,ss])
df.loc[df.data=='sub',CFG.target] = np.nan

df['Tarih'] = pd.to_datetime(df['Tarih'])
df['Tarih_daily'] = pd.to_datetime(df.Tarih.apply(lambda x: x.strftime('%Y-%m-%d')))
df = df.sort_values('Tarih').reset_index(drop=True)
df.head()

,Tarih,Dağıtılan Enerji (MWh),data,Tarih_daily
0,2018-01-01 00:00:00,1593.944216,train,2018-01-01
1,2018-01-01 01:00:00,1513.933887,train,2018-01-01
2,2018-01-01 02:00:00,1402.612637,train,2018-01-01
3,2018-01-01 03:00:00,1278.527266,train,2018-01-01
4,2018-01-01 04:00:00,1220.697701,train,2018-01-01


In [6]:
df.tail()

,Tarih,Dağıtılan Enerji (MWh),data,Tarih_daily
40891,2022-08-31 19:00:00,NaN,sub,2022-08-31
40892,2022-08-31 20:00:00,NaN,sub,2022-08-31
40893,2022-08-31 21:00:00,NaN,sub,2022-08-31
40894,2022-08-31 22:00:00,NaN,sub,2022-08-31
40895,2022-08-31 23:00:00,NaN,sub,2022-08-31


# Utils

In [7]:
def real_time_consumption(start_date="2017-12-01",
                         end_date="2022-10-01"):
    
    """
    Türkiyenin toplam enerji tüketimi. 24 saat sonrasını tahmin etmek için minumum shift(24) ile kullandığım
    EPİAŞ şeffaflık real-time elektrik tüketim verisi.
    Tarih aralığı geniş olduğundan veriyi çekmek uzun sürebilir.
    
    """
    url=f"https://seffaflik.epias.com.tr/transparency/service/consumption/real-time-consumption?startDate={start_date}&endDate={end_date}"
    response=requests.get(url,verify=False)
    json_data=json.loads(response.text.encode('utf8'))
    consumption=pd.DataFrame(json_data['body']['hourlyConsumptions'])
    consumption['Tarih']=pd.to_datetime(consumption.date.str[:16])
    consumption = consumption[['consumption','Tarih']]
    
    return consumption


def real_time_generation(start_date="2017-12-01",
                         end_date="2022-10-01"):
    
    """
    Türkiyenin toplam enerji üretimi. 24 saat sonrasını tahmin etmek için minumum shift(24) ile kullandığım EPİAŞ şeffaflık real-time elektrik üretim verisi.
    Tarih aralığı geniş olduğundan veriyi çekmek uzun sürebilir. Her ihtimale karşın, ekstra data olarak gdz-ext-dataset'de production.csv dosyasında bulunabilir.
    
    """
    url=f"https://seffaflik.epias.com.tr/transparency/service/production/real-time-generation?startDate={start_date}&endDate={end_date}"
    response=requests.get(url,verify=False)
    json_data=json.loads(response.text.encode('utf8'))
    production = pd.DataFrame(json_data['body']['hourlyGenerations'])
    production['Tarih']=pd.to_datetime(production.date.str[:16])
    production.loc[production.total==0,'total'] = np.nan
    production.drop(['date','naphta','nucklear'],axis=1,inplace=True)
    
    return production

def nasa_external_data_preprocessor(df_temp):
    """
    Data şu kaynaktan beslenmektedir: https://power.larc.nasa.gov/data-access-viewer/
    
    Buradan şu lokasyonlara ait:
    İzmir:
        latitude = 38.4235 
        longitude = 27.1564
    Manisa:
        latitude = 38.630554
        longitude = 27.422222
        
    veriler toplanmıştır. Python API olmadığı için indirip data olarak ekledim,
    "csv" formatında gdz-ext-dataset datası içinde bulabilirsiniz.
    Veriler benzer olduğu için Manisa verisi kullanmadım. İzmir hava durumu verisi yeterli oldu.
    
    """
    
    df_temp['Tarih'] = df_temp['YEAR'].astype(str) + "-" \
                    + df_temp['MO'].astype(str) + "-"\
                    + df_temp['DY'].astype(str) + " "\
                    + df_temp['HR'].astype(str) + ":00"
    df_temp = df_temp.drop(['YEAR','MO','DY','HR'],axis=1)
    df_temp['Tarih'] = pd.to_datetime(df_temp['Tarih'])
    return df_temp


def read_meteostat_weather_data(latitude=38.4235,
                                longitude=27.1564,
                                start_date=datetime(2017, 4, 1),
                                end_date=datetime(2023, 12, 31, 23, 59)):
    
    """
    Hava durumu verisi için ikinci bir kaynak. Python API'ı mevcut: https://github.com/meteostat/meteostat-python
    
    National Oceanic and Atmospheric Administration (NOAA) ve Germany's national meteorological service (DWD)
    kaynaklarından beslenmektedir.
    
    Bu veri üzerinde de farklı denemeler yaptım ancak izmir location'ı modele en faydalı olandı. Bu nedenle Manisa'yı
    dahil etmedim.
    """
    
    location = Point(latitude, longitude)
    weather = Hourly('72219', start_date, end_date)
    weather = weather.fetch()
    weather = weather.reset_index().rename(columns = {'time':'Tarih'})
    weather = weather.drop(['snow','wpgt','tsun'],axis=1)
    weather['Tarih'] = pd.to_datetime(weather['Tarih'])
    return weather


def solar_features(city = 'İzmir'):
    
    """
    Kaynak: https://github.com/pvlib/pvlib-python
    
    Güneş ışınlarının ilgili location'a gelme açısını belirten featurelar.
    
    Bu featureları istediğimiz dönem için lead ve lag feature olarak üretebiliriz.
    Herhangi bir leak unsuru yaratmayacaktır. Seasonality etkisini yansıtmak için modele katkısı yüksekti.

    """

    if city == 'İzmir':
        latitude = 38.4235 
        longitude = 27.1564
    if city == 'Manisa':
        latitude = 38.630554
        longitude = 27.422222
        
    tz = 'Europe/Istanbul'
    start='2017-01-01'
    end='2023-01-01'
    tus = Location(latitude, longitude, tz, 700, city)
    
    times = pd.date_range(start, end, freq='H', tz=tus.tz)
    cs = tus.get_clearsky(times)  # ineichen with climatology table by default
    cs=cs.reset_index().rename(columns={'index':'Tarih'})
    cs['Tarih'] = pd.to_datetime(cs['Tarih'], format = "%d/%m/%Y %H/%M")
    cs['Tarih']=cs['Tarih'].dt.tz_localize(None)
    cs=cs.rename(columns={'ghi':f'ghi_{city}','dni':f'dni_{city}','dhi':f'dhi_{city}'})
    
    return cs

def solar_features_advanced(lat=38.4235,
                            lon=27.1564,
                            start='2017-01-01',
                            end='2023-01-01'):
    """
    Kaynak: https://github.com/pvlib/pvlib-python
    
    Güneş ışınlarının ilgili location'a gelme açısını belirten diğer detaylı featurelar.
    
    Seasonality etkisini yansıtmak için modele katkısı yüksekti.
    
    """
    tz = 'Europe/Istanbul'
    tus = Location(lat, lon, tz, 700, 'İzmir')
    
    #lat, lon = latitude, longitude
    times = pd.date_range(start, end, closed='left', freq='H',tz=tz)
    solpos = solarposition.get_solarposition(times, lat, lon)
    truetracking_angles = tracking.singleaxis(
        apparent_zenith=solpos['apparent_zenith'],
        apparent_azimuth=solpos['azimuth'],
        axis_tilt=0,
        axis_azimuth=180,
        max_angle=90,
        backtrack=False,  # for true-tracking
        gcr=0.5)  # irrelevant for true-tracking
    truetracking_position = truetracking_angles['tracker_theta'].fillna(0)
    
    solpos=solpos.reset_index().rename(columns={'index':'Tarih'})
    solpos['Tarih'] = pd.to_datetime(solpos['Tarih'], format = "%d/%m/%Y %H/%M")
    solpos['Tarih']=solpos['Tarih'].dt.tz_localize(None)
    solpos=solpos.drop(['zenith','elevation'],axis=1)
    solpos.drop('apparent_elevation',axis=1,inplace=True)
    
    truetracking_position = truetracking_position.reset_index().rename(columns={'index':'Tarih','tracker_theta':'sun_position'})
    truetracking_position['Tarih'] = pd.to_datetime(truetracking_position['Tarih'], format = "%d/%m/%Y %H/%M")
    truetracking_position['Tarih'] = truetracking_position['Tarih'].dt.tz_localize(None)
    truetracking_position['Tarih'] = pd.to_datetime(truetracking_position['Tarih'])
    
    turbidity = pvlib.clearsky.lookup_linke_turbidity(times, lat, lon, interp_turbidity=True)
    turbidity=turbidity.reset_index().rename(columns={'index':'Tarih',0:'turbidity'})
    turbidity['Tarih'] = pd.to_datetime(turbidity['Tarih'], format = "%d/%m/%Y %H/%M")
    turbidity['Tarih']=turbidity['Tarih'].dt.tz_localize(None)
    
    return solpos, truetracking_position, turbidity

def lag_features(df_temp,
                 columns,
                 lags):
    
    """
    Lag feature üreten fonksiyon, ilgili dataframe, lag uzunlukluları ve sütun isimleri verilerek lag'li featurelar üretilir.
    """
    for col in columns:
        for lag in lags:
            df_temp[f'lag_{lag}_{col}'] = df_temp[col].shift(lag)
    return df_temp


def rolling_features(df_temp,
                     columns,
                     rolls,
                     roll_types):
    """
    Rolling feature üreten fonksiyon, ilgili dataframe, rolling type'ları ve sütun isimleri verilerek rolling featurelar üretilir.
    """
    
    for col in columns:
        for roll in rolls:
            if 'mean' in roll_types:
                df_temp[f'rolling_mean_{roll}_{col}'] = df_temp[col].rolling(roll,min_periods=1).mean().reset_index(drop=True)
            if 'max' in roll_types:
                df_temp[f'rolling_max_{roll}_{col}'] = df_temp[col].rolling(roll,min_periods=1).max().reset_index(drop=True)
            if 'min' in roll_types:
                df_temp[f'rolling_min_{roll}_{col}'] = df_temp[col].rolling(roll,min_periods=1).min().reset_index(drop=True)
            if 'std' in roll_types:
                df_temp[f'rolling_std_{roll}_{col}'] = df_temp[col].rolling(roll,min_periods=1).std().reset_index(drop=True)
    return df_temp


def rolling_shift_features(df_temp,
                     columns,
                     rolls,
                     roll_types):
    """
    24 lagli rolling feature üreten fonksiyon, ilgili dataframe, rolling type'ları ve sütun isimleri verilerek rolling featurelar üretilir.
    """
    for col in columns:
        for roll in rolls:
            if 'mean' in roll_types:
                df_temp[f'rolling_shift_24_mean_{roll}_{col}'] = df_temp[col].shift(24).rolling(roll,min_periods=1).mean().reset_index(drop=True)
            if 'max' in roll_types:
                df_temp[f'rolling_shift_24_max_{roll}_{col}'] = df_temp[col].shift(24).rolling(roll,min_periods=1).max().reset_index(drop=True)
            if 'min' in roll_types:
                df_temp[f'rolling_shift_24_min_{roll}_{col}'] = df_temp[col].shift(24).rolling(roll,min_periods=1).min().reset_index(drop=True)
            if 'std' in roll_types:
                df_temp[f'rolling_shift_24_std_{roll}_{col}'] = df_temp[col].shift(24).rolling(roll,min_periods=1).std().reset_index(drop=True)
    return df_temp


def diff_pct_features(df_temp,
                  columns,
                  diff_pct
                 ):
    """
    Difference ve percentile change featureları üreten fonksiyon.
    
    """
    for col in columns:
        for value in diff_pct:
            df_temp[f'diff_{col}_{value}'] = df_temp[col].diff(value)
            df_temp[f'pct_change_{col}_{value}'] = df_temp[col].pct_change(value)
    return df_temp

def datetime_features(df_temp):
    """
    Datetime feature üretir.
    """
    df_temp['month'] = df_temp['Tarih'].dt.month
    df_temp['hour'] = df_temp['Tarih'].dt.hour
    df_temp['year'] = df_temp['Tarih'].dt.year
    df_temp['dayofweek'] = df_temp['Tarih'].dt.dayofweek
    df_temp['quarter'] = df_temp['Tarih'].dt.quarter
    df_temp['dayofmonth'] = df_temp['Tarih'].dt.day
    df_temp['weekofyear'] = df_temp['Tarih'].dt.weekofyear
    return df_temp

def periodic_spline_transformer(period, n_splines=None, degree=3):
    """
    Kaynak: https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html
    """
    
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True)

def seasonality_features(df_temp):
    df_temp['month_sin'] = np.sin(2*np.pi*df_temp.month/12)
    df_temp['month_cos'] = np.cos(2*np.pi*df_temp.month/12)
    df_temp['day_sin'] = np.sin(2*np.pi*df_temp.hour/24)
    df_temp['day_cos'] = np.cos(2*np.pi*df_temp.hour/24)
    return df_temp

def seasonality_spline_features(hours=np.arange(0,24)):
    hour_df = pd.DataFrame(np.linspace(0, 24, 24).reshape(-1, 1),columns=["hour"])
    splines = periodic_spline_transformer(24, n_splines=12).fit_transform(hour_df)
    splines_df = pd.DataFrame(splines,columns=[f"spline_{i}" for i in range(splines.shape[1])])
    splines_df =pd.concat([pd.Series(hours,name='hour'), splines_df], axis="columns")
    
    return splines_df

def yuk_tahmin_features(path):
    """
    Bir önceden belirlenen yük tahmin planı verisi önişleme fonksiyonu.
    """
    yuk_tahmin_plani = pd.read_csv(path)
    yuk_tahmin_plani['Tarih'] = pd.to_datetime(yuk_tahmin_plani['Tarih'].astype(str) + " " + yuk_tahmin_plani['Saat'].astype(str))
    yuk_tahmin_plani = yuk_tahmin_plani[['Tarih','yuk_tahmin_plani']]
    yuk_tahmin_plani['yuk_tahmin_plani'] = yuk_tahmin_plani['yuk_tahmin_plani'].str.replace(',','').str.replace('.','').astype(float)
    
    return yuk_tahmin_plani

def kgup_features(path):
    """
    Final solutionda kullanılmadı. Kesinleşmiş Günlük Üretim Planı (KGÜP) featureları üretir.
    
    """
    kgup = pd.read_csv(path)
    kgup['Tarih'] = pd.to_datetime(kgup['Tarih'].astype(str) + " " + kgup['Saat'].astype(str))
    kgup.drop('Saat',axis=1,inplace=True)
    kgup = kgup.set_index('Tarih')
    kgup = kgup.add_prefix('kgup_')
    for col in kgup.columns:
        kgup[col] = kgup[col].str.replace(',','').str.replace('.','').astype(float)
    kgup = kgup.reset_index()
    return kgup

def holiday_features(path):
    """
    Tatil günlerinden türetilmiş featureları üretir.
    
    """
    
    hol = pd.read_csv(path, parse_dates=['CALENDAR_DATE'])
    hol = hol[['CALENDAR_DATE','WEEKEND_FLAG','RAMADAN_FLAG','RELIGIOUS_DAY_FLAG_SK','NATIONAL_DAY_FLAG_SK','PUBLIC_HOLIDAY_FLAG']].rename(columns={'CALENDAR_DATE':'Tarih_daily'})
    hol = hol.sort_values('Tarih_daily').reset_index(drop=True)
    hol.loc[hol.RELIGIOUS_DAY_FLAG_SK != 100, 'RELIGIOUS_DAY_FLAG_SK'] = 1
    hol.loc[hol.RELIGIOUS_DAY_FLAG_SK == 100, 'RELIGIOUS_DAY_FLAG_SK'] = 0
    
    hol.loc[hol.NATIONAL_DAY_FLAG_SK != 200, 'NATIONAL_DAY_FLAG_SK'] = 1
    hol.loc[hol.NATIONAL_DAY_FLAG_SK == 200, 'NATIONAL_DAY_FLAG_SK'] = 0
    
    hol.loc[hol.RAMADAN_FLAG == 'N', 'RAMADAN_FLAG'] = 0
    hol.loc[hol.RAMADAN_FLAG == 'Y', 'RAMADAN_FLAG'] = 1
    
    hol.loc[hol.PUBLIC_HOLIDAY_FLAG == 'N', 'PUBLIC_HOLIDAY_FLAG'] = 0
    hol.loc[hol.PUBLIC_HOLIDAY_FLAG == 'Y', 'PUBLIC_HOLIDAY_FLAG'] = 1
    
    hol.loc[hol.WEEKEND_FLAG == 'N', 'WEEKEND_FLAG'] = 0
    hol.loc[hol.WEEKEND_FLAG == 'Y', 'WEEKEND_FLAG'] = 1
    
    hol['WEEKEND_FLAG'] = hol['WEEKEND_FLAG'].astype(int)
    hol['RAMADAN_FLAG'] = hol['RAMADAN_FLAG'].astype(int)
    hol['PUBLIC_HOLIDAY_FLAG'] = hol['PUBLIC_HOLIDAY_FLAG'].astype(int)
    # Resmi/Dini/Milli Bayram ve Tatilleri önceden bildiren featurelar
    is_next_days_cols = ['RAMADAN_FLAG','PUBLIC_HOLIDAY_FLAG','RELIGIOUS_DAY_FLAG_SK','NATIONAL_DAY_FLAG_SK']
    
    for i in [3,7,15]:
        for col in is_next_days_cols:
            hol[f"is_{col}_in_next_{i}_days"] = hol[col].rolling(i).sum().shift(-i)

    #hol = hol.set_index('Tarih_daily')
    #how_many_days_left_columns = ['WEEKEND_FLAG','RAMADAN_FLAG','RELIGIOUS_DAY_FLAG_SK','NATIONAL_DAY_FLAG_SK','PUBLIC_HOLIDAY_FLAG']
    #for col in how_many_days_left_columns:
    #    dates = hol[hol[col]==1].index.tolist()
    #    s = pd.Series(hol.index.isin(dates), index=hol.index)[::-1].cumsum()
    #    hol[f'days_left_next_{col}'] = hol.groupby(s).cumcount(ascending=False)# + 1
    #    
    #hol = hol.reset_index()
    
    return hol
    

### Read and Preprocess External Datasets

In [8]:
# holiday data for Turkey. Kaynak: https://www.kaggle.com/datasets/frtgnn/turkish-calendar
if CFG.holiday_features:
    hol = holiday_features(CFG.holiday_path)

# merge NASA weather data
if CFG.nasa_weather_features:
    izmir_temp = nasa_external_data_preprocessor(pd.read_csv(r'C:\Users\Emincan\Desktop\Competition-Ensembles\Gdz\1st-Solution\externals\ext_izmir_temp.csv')) # Temperature
    izmir_hum = nasa_external_data_preprocessor(pd.read_csv(r'C:\Users\Emincan\Desktop\Competition-Ensembles\Gdz\1st-Solution\externals\ext_izmir_humi.csv')) # Humidity
    izmir_wind = nasa_external_data_preprocessor(pd.read_csv(r'C:\Users\Emincan\Desktop\Competition-Ensembles\Gdz\1st-Solution\externals\ext_izmir_wind.csv')) # Wind
    izmir_nasa = izmir_temp.merge(izmir_hum.merge(izmir_wind, how='left', on='Tarih'),how='left',on='Tarih')
    del izmir_temp,izmir_hum,izmir_wind; gc.collect()

# weather data: meteostat
if CFG.meteostat_weather_features:
    izmir_meteostat = read_meteostat_weather_data()

#solar features
if CFG.solar_features:
    cs_izmir = solar_features("İzmir")
    #cs_man = solar_features("Manisa")
    solpos,truetracking_position,turbidity = solar_features_advanced()

#lagged real time consumption data
if CFG.consumption_lag_features:
    consumption = real_time_consumption()
    
#lagged real time generation data
if CFG.production_lag_features:
    if CFG.read_from_path:
        production = pd.read_csv(r'C:\Users\Emincan\Desktop\Competition-Ensembles\Gdz\1st-Solution\externals\production.csv')
        production.drop(['naphta','nucklear'],axis=1,inplace=True)
        production['Tarih'] = pd.to_datetime(production['Tarih'])
        production.loc[production.total==0,'total'] = np.nan
    if CFG.read_from_path == False:
        production = real_time_generation()

### Feature Engineering

In [9]:
#datetime features
df = datetime_features(df)

In [10]:
if CFG.holiday_features:
    df = df.merge(hol,how='left',on='Tarih_daily')

In [11]:
if CFG.seasonality_features:
    df = seasonality_features(df)

In [12]:
if CFG.seasonality_spline_features:
    splines_df = seasonality_spline_features()
    df.merge(splines_df,on='hour',how='left')

In [13]:
if CFG.nasa_weather_features:
    if CFG.weather_lag_features:
        izmir_nasa = lag_features(izmir_nasa,
                                  columns = CFG.nasa_feature_columns,
                                  lags = CFG.weather_lag_range
                                 )
    if CFG.diff_pct_features:
        izmir_nasa = diff_pct_features(izmir_nasa,
                                   columns = CFG.nasa_feature_columns,
                                   diff_pct = [1]
                                  )
    if CFG.rolling_features:
        izmir_nasa = rolling_features(izmir_nasa,
                                      columns = CFG.nasa_feature_columns,
                                      rolls = CFG.rolling_range,
                                      roll_types=CFG.roll_types
                                     )
    izmir_nasa =izmir_nasa.set_index('Tarih').add_prefix('izmir_').reset_index()
        
if CFG.meteostat_weather_features:
    #if CFG.weather_lag_features:
    #    izmir_meteostat = lag_features(izmir_meteostat,
    #                              columns = CFG.meteostat_feature_columns,
    #                              lags = CFG.weather_lag_range
    #                             )
    #if CFG.diff_pct_features:
    #    izmir_meteostat = diff_pct_features(izmir_meteostat,
    #                               columns = CFG.meteostat_feature_columns,
    #                               diff_pct = [1,6,12,24]
    #                              ) 
    #if CFG.rolling_features:
    #    izmir_meteostat = rolling_features(izmir_meteostat,
    #                                  columns = CFG.meteostat_feature_columns,
    #                                  rolls = CFG.rolling_range,
    #                                  roll_types=CFG.roll_types
    #                                 )
    izmir_meteostat =izmir_meteostat.set_index('Tarih').add_prefix('izmir_').reset_index()

In [14]:
if CFG.solar_features:
    
    if CFG.solar_lag_features:
    
        cs_izmir = lag_features(cs_izmir,
                                columns = ['ghi_İzmir','dni_İzmir','dhi_İzmir'],
                                lags = np.arange(-10,11)
                               )

In [15]:
if CFG.production_features:
    
    if CFG.production_lag_features:
        production = lag_features(production,
                                  columns = CFG.production_base_columns,
                                  lags = [24,25])
    if CFG.rolling_shift_features:
        production = rolling_shift_features(production,
                                            columns = CFG.production_base_columns,
                                            rolls = CFG.rolling_range,
                                            roll_types=CFG.roll_types)
    production.drop(CFG.production_base_columns,axis=1,inplace=True)
    
if CFG.consumption_features:
    
    if CFG.consumption_lag_features:
        consumption = lag_features(consumption,
                                  columns = CFG.consumption_base_columns,
                                  lags = [24,25])
    consumption.drop(CFG.consumption_base_columns,axis=1,inplace=True)

### Merge Features to Main DF

In [16]:
if CFG.meteostat_weather_features:
    df = df.merge(izmir_meteostat,on='Tarih',how='left')
    
if CFG.nasa_weather_features:
    df = df.merge(izmir_nasa,on='Tarih',how='left')
    
if CFG.solar_features:
    df = df.merge(cs_izmir,on='Tarih',how='left')
    df = pd.merge(df,turbidity,on='Tarih',how='left')
    df = pd.merge(df,solpos,on='Tarih',how='left')
    df = pd.merge(df,truetracking_position,on='Tarih',how='left')

if CFG.production_features:
    production = production.set_index('Tarih').add_prefix('production_').reset_index()
    df = pd.merge(df,production,on='Tarih',how='left')
    
if CFG.consumption_features:
    df = pd.merge(df,consumption,on='Tarih',how='left')
    
if CFG.yuk_tahmin_plani_features:
    yuk_tahmin_plani = yuk_tahmin_features(r'C:\Users\Emincan\Desktop\Competition-Ensembles\Gdz\1st-Solution\externals\yuk_tahmin_plani.csv')
    df = df.merge(yuk_tahmin_plani,on='Tarih',how='left')
    
if CFG.kgup_features:
    kgup = kgup_features(r'C:\Users\Emincan\Desktop\Competition-Ensembles\Gdz\1st-Solution\externals\ext_kgp.csv')
    df = df.merge(kgup,on='Tarih',how='left')

In [17]:
df = df.drop([col for col in df.columns if "lag_0" in col],axis=1)
df = df.drop([col for col in df.columns if "rolling_0" in col],axis=1)

In [18]:
df.shape

(40896, 252)

In [19]:
df

,Tarih,Dağıtılan Enerji (MWh),data,Tarih_daily,month,hour,year,dayofweek,quarter,dayofmonth,weekofyear,WEEKEND_FLAG,RAMADAN_FLAG,RELIGIOUS_DAY_FLAG_SK,NATIONAL_DAY_FLAG_SK,PUBLIC_HOLIDAY_FLAG,is_RAMADAN_FLAG_in_next_3_days,is_PUBLIC_HOLIDAY_FLAG_in_next_3_days,is_RELIGIOUS_DAY_FLAG_SK_in_next_3_days,is_NATIONAL_DAY_FLAG_SK_in_next_3_days,is_RAMADAN_FLAG_in_next_7_days,is_PUBLIC_HOLIDAY_FLAG_in_next_7_days,is_RELIGIOUS_DAY_FLAG_SK_in_next_7_days,is_NATIONAL_DAY_FLAG_SK_in_next_7_days,is_RAMADAN_FLAG_in_next_15_days,is_PUBLIC_HOLIDAY_FLAG_in_next_15_days,is_RELIGIOUS_DAY_FLAG_SK_in_next_15_days,is_NATIONAL_DAY_FLAG_SK_in_next_15_days,month_sin,month_cos,day_sin,day_cos,izmir_temp,izmir_dwpt,izmir_rhum,izmir_prcp,izmir_wdir,izmir_wspd,izmir_pres,izmir_coco,izmir_T2M,izmir_T2MDEW,izmir_T2MWET,izmir_QV2M,izmir_RH2M,izmir_PRECTOTCORR,izmir_PS,izmir_WS10M,izmir_WD10M,izmir_WS50M,izmir_lag_5_T2M,izmir_lag_10_T2M,izmir_lag_15_T2M,izmir_lag_20_T2M,izmir_lag_25_T2M,izmir_lag_30_T2M,izmir_lag_35_T2M,izmir_lag_40_T2M,izmir_lag_45_T2M,izmir_lag_50_T2M,izmir_lag_5_T2MDEW,izmir_lag_10_T2MDEW,izmir_lag_15_T2MDEW,izmir_lag_20_T2MDEW,izmir_lag_25_T2MDEW,izmir_lag_30_T2MDEW,izmir_lag_35_T2MDEW,izmir_lag_40_T2MDEW,izmir_lag_45_T2MDEW,izmir_lag_50_T2MDEW,izmir_lag_5_T2MWET,izmir_lag_10_T2MWET,izmir_lag_15_T2MWET,izmir_lag_20_T2MWET,izmir_lag_25_T2MWET,izmir_lag_30_T2MWET,izmir_lag_35_T2MWET,izmir_lag_40_T2MWET,izmir_lag_45_T2MWET,izmir_lag_50_T2MWET,izmir_lag_5_QV2M,izmir_lag_10_QV2M,izmir_lag_15_QV2M,izmir_lag_20_QV2M,izmir_lag_25_QV2M,izmir_lag_30_QV2M,izmir_lag_35_QV2M,izmir_lag_40_QV2M,izmir_lag_45_QV2M,izmir_lag_50_QV2M,izmir_lag_5_RH2M,izmir_lag_10_RH2M,izmir_lag_15_RH2M,izmir_lag_20_RH2M,izmir_lag_25_RH2M,izmir_lag_30_RH2M,izmir_lag_35_RH2M,izmir_lag_40_RH2M,izmir_lag_45_RH2M,izmir_lag_50_RH2M,izmir_lag_5_PRECTOTCORR,izmir_lag_10_PRECTOTCORR,izmir_lag_15_PRECTOTCORR,izmir_lag_20_PRECTOTCORR,izmir_lag_25_PRECTOTCORR,izmir_lag_30_PRECTOTCORR,izmir_lag_35_PRECTOTCORR,izmir_lag_40_PRECTOTCORR,izmir_lag_45_PRECTOTCORR,izmir_lag_50_PRECTOTCORR,izmir_lag_5_PS,izmir_lag_10_PS,izmir_lag_15_PS,izmir_lag_20_PS,izmir_lag_25_PS,izmir_lag_30_PS,izmir_lag_35_PS,izmir_lag_40_PS,izmir_lag_45_PS,izmir_lag_50_PS,izmir_lag_5_WS10M,izmir_lag_10_WS10M,izmir_lag_15_WS10M,izmir_lag_20_WS10M,izmir_lag_25_WS10M,izmir_lag_30_WS10M,izmir_lag_35_WS10M,izmir_lag_40_WS10M,izmir_lag_45_WS10M,izmir_lag_50_WS10M,izmir_lag_5_WD10M,izmir_lag_10_WD10M,izmir_lag_15_WD10M,izmir_lag_20_WD10M,izmir_lag_25_WD10M,izmir_lag_30_WD10M,izmir_lag_35_WD10M,izmir_lag_40_WD10M,izmir_lag_45_WD10M,izmir_lag_50_WD10M,izmir_lag_5_WS50M,izmir_lag_10_WS50M,izmir_lag_15_WS50M,izmir_lag_20_WS50M,izmir_lag_25_WS50M,izmir_lag_30_WS50M,izmir_lag_35_WS50M,izmir_lag_40_WS50M,izmir_lag_45_WS50M,izmir_lag_50_WS50M,ghi_İzmir,dni_İzmir,dhi_İzmir,lag_-10_ghi_İzmir,lag_-9_ghi_İzmir,lag_-8_ghi_İzmir,lag_-7_ghi_İzmir,lag_-6_ghi_İzmir,lag_-5_ghi_İzmir,lag_-4_ghi_İzmir,lag_-3_ghi_İzmir,lag_-2_ghi_İzmir,lag_-1_ghi_İzmir,lag_1_ghi_İzmir,lag_2_ghi_İzmir,lag_3_ghi_İzmir,lag_4_ghi_İzmir,lag_5_ghi_İzmir,lag_6_ghi_İzmir,lag_7_ghi_İzmir,lag_8_ghi_İzmir,lag_9_ghi_İzmir,lag_10_ghi_İzmir,lag_-10_dni_İzmir,lag_-9_dni_İzmir,lag_-8_dni_İzmir,lag_-7_dni_İzmir,lag_-6_dni_İzmir,lag_-5_dni_İzmir,lag_-4_dni_İzmir,lag_-3_dni_İzmir,lag_-2_dni_İzmir,lag_-1_dni_İzmir,lag_1_dni_İzmir,lag_2_dni_İzmir,lag_3_dni_İzmir,lag_4_dni_İzmir,lag_5_dni_İzmir,lag_6_dni_İzmir,lag_7_dni_İzmir,lag_8_dni_İzmir,lag_9_dni_İzmir,lag_10_dni_İzmir,lag_-10_dhi_İzmir,lag_-9_dhi_İzmir,lag_-8_dhi_İzmir,lag_-7_dhi_İzmir,lag_-6_dhi_İzmir,lag_-5_dhi_İzmir,lag_-4_dhi_İzmir,lag_-3_dhi_İzmir,lag_-2_dhi_İzmir,lag_-1_dhi_İzmir,lag_1_dhi_İzmir,lag_2_dhi_İzmir,lag_3_dhi_İzmir,lag_4_dhi_İzmir,lag_5_dhi_İzmir,lag_6_dhi_İzmir,lag_7_dhi_İzmir,lag_8_dhi_İzmir,lag_9_dhi_İzmir,lag_10_dhi_İzmir,turbidity,apparent_zenith,azimuth,equation_of_time,sun_position,production_lag_24_fueloil,production_lag_25_fueloil,production_lag_24_gasOil,production_lag_25_gasOil,production_lag_24_black

### Modelling

#### Modelleme Stratejisi
* Validasyon: Son 3 ayı içeren Expanding Window Time-Series Split Validation. Bu 3 ayı seçme nedenim bayramları ve özel günleri içeren geniş bir zaman aralığı olması. Böylece yalnızca ağustos için değil diğer aylar için de performansı karşılaştırabiliriz.
* 2 Step Model:
    * 1. Aşama EPIAŞ şeffaflık platformundan alınan minumum lag(24)'lü featurelarla train
    * 2. Aşama EPIAŞ verilerini kullanmadan train
* Nihai olarak ise bu iki yaklaşımın ensemble alınmış hali:
    * Bu yaklaşımın her biri LB'de oldukça iyi skor almakta ancak EPIAŞ featurelarını lag'li kullanmak zorunda olduğumuz için modele bazı noktalarda zarar verebiliyor (overfit). Bu nedenle diğer featureların etkisinide yansıtmak için bu iki yaklaşımı birleştirdim.
* Tek bir training fonksiyonu üzerinden farklı feature subsetler ile training yapılabilir. EPIAŞ featurelarında aksama olduğunda dahi conditional parametre konularak EPIAŞ olmayan model sağlıklı bir şekilde infer edilebilir.

In [20]:
cat_features_hol = ['WEEKEND_FLAG',
                    'RAMADAN_FLAG',
                    'RELIGIOUS_DAY_FLAG_SK',
                    'NATIONAL_DAY_FLAG_SK',
                    'PUBLIC_HOLIDAY_FLAG']

cat_features_date = ['month',
                     'hour',
                     'dayofweek']

cat_features = cat_features_date + cat_features_hol

for col in cat_features:
    df[col] = df[col].astype('category')
    
stage_one_exclude = []
stage_two_exclude = [col for col in df.columns if 'production' in col] + \
                    [col for col in df.columns if 'consumption' in col]

In [21]:
stage_two_exclude

['production_lag_24_fueloil',
 'production_lag_25_fueloil',
 'production_lag_24_gasOil',
 'production_lag_25_gasOil',
 'production_lag_24_blackCoal',
 'production_lag_25_blackCoal',
 'production_lag_24_lignite',
 'production_lag_25_lignite',
 'production_lag_24_geothermal',
 'production_lag_25_geothermal',
 'production_lag_24_naturalGas',
 'production_lag_25_naturalGas',
 'production_lag_24_river',
 'production_lag_25_river',
 'production_lag_24_dammedHydro',
 'production_lag_25_dammedHydro',
 'production_lag_24_lng',
 'production_lag_25_lng',
 'production_lag_24_biomass',
 'production_lag_25_biomass',
 'production_lag_24_importCoal',
 'production_lag_25_importCoal',
 'production_lag_24_asphaltiteCoal',
 'production_lag_25_asphaltiteCoal',
 'production_lag_24_wind',
 'production_lag_25_wind',
 'production_lag_24_sun',
 'production_lag_25_sun',
 'production_lag_24_importExport',
 'production_lag_25_importExport',
 'production_lag_24_wasteheat',
 'production_lag_25_wasteheat',
 'producti

In [22]:
train_df = df[df.data=='train']
sub_df = df[df.data=='sub']
exclude_cols = ['Tarih','data','year','Tarih_daily',CFG.target]

sub_df = sub_df.drop(exclude_cols,axis=1)
y = train_df[CFG.target]
X = train_df.drop(exclude_cols,axis=1)

stage_one_features = X.drop(stage_one_exclude,axis=1).columns.tolist()
stage_two_features = X.drop(stage_two_exclude,axis=1).columns.tolist()
len(stage_one_features),len(stage_two_features)

(247, 213)

In [23]:
params = {'learning_rate': 0.03,
          'objective':'MAE',
          'depth': 6,
          'early_stopping_rounds':1000,
          'iterations': 10000,
          'use_best_model': True,
          'eval_metric': "MAPE",
          'random_state': 986,
          'allow_writing_files': False,
          'thread_count':24,
        #   'task_type':'GPU',
        #   'devices':'0'
          }

In [24]:
def catboost_trainer(X,
                     y,
                     submission_df,
                     cv,
                     model_params,
                     feature_list,
                     cat_features,
                     scorer,
                     target_transform=False):
    """
    Catboost Trainer.
    
    ---------
    :param X: training data
    :param y: target
    :param submission_df: test dataframe to be predicted
    :param cv: scikitlearn cross validation object
    :param model_params: dict of catboost model parameters
    :param feature_list: used list of features for training and inference
    :param cat_features: categorical features
    :param scorer: scikitlearn evaluation metric
    :param target_transform: Target
    :return: CV score list, models and submission predictions 
    """
    
    score_list = []
    fold = 1
    unseen_preds = []
    importance = []
    val_results = []
    train_results = []
    models = []
    
    for train_index, test_index in cv.split(X):
        X_train,X_val = X.iloc[train_index][feature_list],X.iloc[test_index][feature_list]
        y_train,y_val = y.iloc[train_index],y.iloc[test_index]
        sub_df_subset = submission_df[feature_list]
        print(f"Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}")
        
        if target_transform:
            y_train = np.log1p(y_train)
            y_val = np.log1p(y_val)
        
        
        model = CatBoostRegressor(**model_params,
                                cat_features=cat_features
                               )
        model.fit(X_train,y_train,
                eval_set=[(X_val,y_val)],
                verbose=500)
        models.append(model)
        forecast_pred = model.predict(sub_df_subset)
        if target_transform:
            forecast_pred = np.expm1(forecast_pred)
        unseen_preds.append(forecast_pred)
        
        val_result = model.predict(X_val)
        if target_transform:
            val_result = np.expm1(val_result)
        
        train_result = model.predict(X_train)
        if target_transform:
            train_result = np.expm1(train_result)
            
        train_results.append(train_result)
        if target_transform:    
            y_train = np.expm1(y_train)
            y_val = np.expm1(y_val)
            
        score = scorer(y_val,val_result)
        score_t = scorer(y_train,train_result)
        
        print(f"Score FOLD-{fold}:{score}")
        print(f"Score Train FOLD-{fold}:{score_t}")
        print(f'Predicted Mean:{np.mean(forecast_pred)}')
        score_list.append(score)
        importance.append(model.get_feature_importance())
        fold += 1
        print('*'*50)
    print("Mean MAPE:", np.mean(score_list),"Std MAPE:",np.std(score_list))
    return score_list, models, unseen_preds

In [25]:
score_list_stage_one_cat, models_stage_one_cat, unseen_preds_stage_one_cat = catboost_trainer(X=X,
                                                                                      y=y,
                                                                                      submission_df=sub_df,
                                                                                      cv=TimeSeriesSplit(n_splits=3,test_size=744),
                                                                                      model_params=params,
                                                                                      feature_list=stage_one_features,
                                                                                      cat_features=cat_features,
                                                                                      scorer=mean_absolute_percentage_error,
                                                                                      target_transform=False)

Training data shape: (37920, 247), Validation data shape: (744, 247)
0:	learn: 0.1934423	test: 0.2486065	best: 0.2486065 (0)	total: 184ms	remaining: 30m 37s
500:	learn: 0.0270912	test: 0.0339840	best: 0.0337616 (498)	total: 16.9s	remaining: 5m 19s
1000:	learn: 0.0206167	test: 0.0306353	best: 0.0306353 (1000)	total: 33s	remaining: 4m 56s
1500:	learn: 0.0174863	test: 0.0297538	best: 0.0297360 (1492)	total: 49.1s	remaining: 4m 38s
2000:	learn: 0.0156230	test: 0.0290875	best: 0.0290797 (1993)	total: 1m 5s	remaining: 4m 20s
2500:	learn: 0.0142790	test: 0.0285564	best: 0.0285564 (2500)	total: 1m 21s	remaining: 4m 2s
3000:	learn: 0.0132596	test: 0.0283053	best: 0.0283053 (3000)	total: 1m 36s	remaining: 3m 46s
3500:	learn: 0.0124465	test: 0.0281902	best: 0.0281566 (3289)	total: 1m 52s	remaining: 3m 29s
4000:	learn: 0.0117599	test: 0.0277745	best: 0.0277694 (3970)	total: 2m 8s	remaining: 3m 13s
4500:	learn: 0.0111952	test: 0.0275807	best: 0.0275807 (4500)	total: 2m 24s	remaining: 2m 56s
5000:	l

In [26]:
score_list_stage_two_cat, models_stage_two_cat, unseen_preds_stage_two_cat = catboost_trainer(X=X,
                                                                                      y=y,
                                                                                      submission_df=sub_df,
                                                                                      cv=TimeSeriesSplit(n_splits=3,test_size=744),
                                                                                      model_params=params,
                                                                                      feature_list=stage_two_features,
                                                                                      cat_features=cat_features,
                                                                                      scorer=mean_absolute_percentage_error,
                                                                                      target_transform=False)

Training data shape: (37920, 213), Validation data shape: (744, 213)
0:	learn: 0.1937587	test: 0.2489154	best: 0.2489154 (0)	total: 30.2ms	remaining: 5m 2s
500:	learn: 0.0348276	test: 0.0408763	best: 0.0408115 (489)	total: 16.5s	remaining: 5m 13s
1000:	learn: 0.0270754	test: 0.0380209	best: 0.0380168 (999)	total: 32.5s	remaining: 4m 52s
1500:	learn: 0.0231448	test: 0.0366230	best: 0.0365859 (1476)	total: 48.2s	remaining: 4m 33s
2000:	learn: 0.0205335	test: 0.0365194	best: 0.0363642 (1895)	total: 1m 3s	remaining: 4m 14s
2500:	learn: 0.0187456	test: 0.0360351	best: 0.0359650 (2443)	total: 1m 19s	remaining: 3m 57s
3000:	learn: 0.0173773	test: 0.0357103	best: 0.0357051 (2999)	total: 1m 34s	remaining: 3m 40s
3500:	learn: 0.0162336	test: 0.0353612	best: 0.0353580 (3483)	total: 1m 49s	remaining: 3m 24s
4000:	learn: 0.0153626	test: 0.0352211	best: 0.0352203 (3998)	total: 2m 5s	remaining: 3m 7s
4500:	learn: 0.0145851	test: 0.0351512	best: 0.0351434 (4494)	total: 2m 20s	remaining: 2m 51s
5000:	l

### Feature Importance

#### Stage 1 Model Feature Importance

In [27]:
importance = [model.get_feature_importance() for model in models_stage_one_cat]

f_importance = pd.concat([pd.Series(X[stage_one_features].columns.to_list(),name='Feature'),
                          pd.Series(np.mean(importance,axis=0),name="Importance")],
                         axis=1).sort_values(by='Importance',
                                             ascending=True)

fig = px.bar(f_importance.tail(20),x='Importance',y='Feature')
fig.update_layout(
    title_text="First 20 Important Features - CatBoost Average of Folds"
)
fig.show()

#### Stage 2 Model Feature Importance

In [28]:
importance = [model.get_feature_importance() for model in models_stage_two_cat]

f_importance = pd.concat([pd.Series(X[stage_two_features].columns.to_list(),name='Feature'),
                          pd.Series(np.mean(importance,axis=0),name="Importance")],
                         axis=1).sort_values(by='Importance',
                                             ascending=True)

fig = px.bar(f_importance.tail(20),x='Importance',y='Feature')
fig.update_layout(
    title_text="First 20 Important Features - CatBoost Average of Folds"
)
fig.show()

#### Make Submission

In [29]:
ss[CFG.target] = np.mean(np.concatenate([unseen_preds_stage_one_cat,unseen_preds_stage_two_cat]),axis=0)

In [30]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=ss.Tarih,y=ss[CFG.target],mode='lines'))

In [ ]:
# ss[['Tarih',CFG.target]].to_csv('submission.csv',index=False)

In [ ]:
# Done.